First, create the model. This must match the model used in the interactive training notebook.

In [1]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

/home/snell/pynis/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/snell/pynis/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/snell/pynis/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Next, load the saved model.  Enter the model path you used to save.

In [ ]:
model.load_state_dict(torch.load('lane_following_model_trt.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
torch.save(model_trt.state_dict(), 'lane_following_model_trt.pth')

Load the optimized model by executing the cell below

In [2]:
import torch
import torchvision
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('lane_following_model_trt.pth')) #'lane_following_trt.pth'))

<All keys matched successfully>

Create the racecar class

In [9]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Connect to the camera

In [4]:
import cv2
camera = cv2.VideoCapture(0)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [8]:
car.throttle = 0.1 # Stop the car

Run the following to start up the car in self-driving mode. 

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 1
STEERING_BIAS = -0.1

car.throttle = -0.01

width = 224
height = 224
while True:
    _, image = camera.read()
    image = cv2.resize(image, (int(width),int(height)))
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    car.steering = x * STEERING_GAIN + STEERING_BIAS

*Note*: Restart the above cell after modifying it. Running it again while it is running will not work.